In [45]:
import os
import xml.etree.ElementTree
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

separar en sentences
post tagging
lematizar
eliminar stop words

todo esto por documento

'AP881218-0003'

In [ ]:
inputXML = './data/ap/ap.txt'
e = xml.etree.ElementTree.parse(inputXML).getroot()
docs = {}

for elem in e.getchildren():
    text = elem.find('TEXT').text
    docno = elem.find('DOCNO').text.strip()
    docs[docno] = text

docs

In [7]:
sentences_by_doc = {}

for docno, text in docs.items():
    sentences_by_doc[docno] = nltk.sent_tokenize(text)


In [36]:
tags_by_doc = {}
for docno, sentences in sentences_by_doc.items():
    tagged_words = {}
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        tags = nltk.pos_tag(tokens)
        for tag in tags:
            tag_key = tag[1]
            tag_value = tag[0]
            tagged_words.setdefault(tag_key, [])
            if (tag_value not in tagged_words[tag_key]):
                tagged_words[tag_key].append(tag_value)
                             
    tags_by_doc[docno] = tagged_words
    

In [56]:
from nltk.corpus import wordnet

def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [57]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('are', get_wordnet_pos('V'))


lemmas_by_doc = {}
for docno, tagged_words in tags_by_doc.items():
    lemmas = set()
    for tag, words in tagged_words.items():
        wornet_pos = get_wordnet_pos(tag)
        for word in words:
            if wornet_pos:
                lemma = lemmatizer.lemmatize(word, wornet_pos)
            else:
                lemma = lemmatizer.lemmatize(word)
            lemmas.add(lemma)
    lemmas_by_doc[docno] = lemmas
    


In [70]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

words_by_doc = {}
for docno, lemmas in lemmas_by_doc.items():
    words_by_doc[docno] = lemmas - stop_words
    
#words_by_doc['AP881218-0003']

In [87]:
import functools

vocabulary = sorted(functools.reduce(lambda x,y: x | y, words_by_doc.values()))

In [89]:
print("El tamaño del vocabulario es", len(vocabulary))

El tamaño del vocabulario es 41191


## 2)

In [102]:
from nltk.tokenize import wordpunct_tokenize

punctuation = set(string.punctuation)
tokens = []
with open('./data/on-the-origin-of-species.txt') as file:  
    data = file.read() 
    tokens = set(wordpunct_tokenize(data)) - punctuation